# Summary

В статье написано, как из набора кластеров получить их интерпретацию. Без линейных комбинаций, без нечетких формулировок - только понятный всем набор порогов и все. Результат приведен на рисунках ниже
- Таблица с порогами 
- PCA с визуализацией

# Кластеризация

Кластеризация - очень полезный способ превращения данных в инсайты и генерации классных гипотез. Она позволяет объединить те наблюдения о вашем продукте, которые ранее выглядели хаотичным набором точек. 

Вот только стоит вспомнить, что выходом модели кластеризации по определению является только соответствие наблюдений и ближайших к ним скоплений. 

Конечно же, сразу приходят на ум мульены способов интерпретации кластеров, но к наиболее популярным из них есть вопросы:
- Иногда достаточно знать центры скоплений, которые легко посчитать. Но вряд ли ваш заказчик захочет проводить кампанию среди клиентов, сделавших примерно 23 заказа за последний месяц. А "примерно 23" - это от 22 до 24, или от 10 до 30? 
- Иногда можно применить техники снижения размерности - например, PCA. Но я неплохо так напрягся, объясняя на пальцах одному щепетильному заказчику выражение "2 * log(дни активности - 10) - 3.2 * log(минуты игры)". А ведь это самое обыкновенное выражение для значений, которые попадают на красивую финальную визуализацию


Наверное, можно всего-то найти несколько простых правил, например: "99% пользователей кластера  купили более 40 шмубликов". И потом для кластеров, которые подчиняются правилу, можно было бы придумать короткое описание - например "фанаты шмубликов". Изи-пизи!

Как же это сделать?

# Explanatory algorithm

Для начала давай определим граф сравнений $G = (V, E)$. Это ориентированный граф, все его вершины будут соответствовать кластерам. Каждой дуге графа назначим еще три дополнительных атрибута - признак f, доля p и порог отсечения v. Два кластера будет связывать дуга в том случае, если большая часть значений некоторого признака в первом кластере ниже порога, а такая же часть этого же признака во втором кластере выше того же порога. Формально и с большим количеством подробностей условие наличия дуги можно записать так:
$$(c_1, c_2, f, v, p) \in E = Q(f(c_1), p) \leq v \land Q(f(c_1), 1 - p) > v $$

(Добавить условие минимальности порога)

Здесь $Q(s, p)$ - квантиль уровня p значений множества s, а $f(c)$ - множество значений признака $f$ элементов кластера $c$. 

Мы можем теперь объединять дуги графа $G$ в множества $S_i$. Каждому множеству $S_i$ соответствует $S'_i$ - множество неориентированных ребер, соединяющих вершины $V$ в случае, если между ними есть хотя бы одна дуга в множестве $S_i$. 

Некоторым $S_i$ также соответствует некоторое правило, которое станет частью интерпретации. Например, множеству дуг следующего вида:

$$(c_1, c_2, session\_time, 14\ min, 2\%)\\(c_3, c_2, session\_time, 9\ min, 1\%)\\(c_3, c_4, session\_time, 11\ min, 5\%)$$ 

может соответствовать правило 

$$((c_1, c_3), [c_2, c_4], session\_time, 9\ min, 1\%)$$

Вот пример рассуждения для одного из кластеров:
- Менее 2% от c1 проводит в приложении больше 14 минут - а значит, точно больше 9 минут 
- Менее 2% от c1 проводит в приложении больше 14 минут - а значит, менее 1% тоже играют больше 14 минут

- Менее 2% от c2 проводит в приложении меньше 14 минут - а значит, точно меньше 9 минут - неверно!!!
- Менее 2% от c2 проводит в приложении меньше 14 минут - а значит, менее 1% тоже играют меньше 14 минут

Похожие рассуждения можно провести и для остальных кластеров. 

Я не буду пока приводить здесь детали реализации алгоритма поиска правил для всех $S_i$ - она громоздкая и требует доработки. Тем не менее, такой набор условий получается сформировать достаточно быстро - за время $O(n \log n)$ (уточнить).

Если теперь мы найдем покрытие полного графа $K_{|V|}$ множествами $S'_i$ то мы найдем набор правил, который отличит каждый кластер от всех остальных. Это и будет искомой интерпретацией, и каждому найденному правилу можно сопоставить короткое текстовое описание. К сожалению, поиск реберного покрытия требует полного перебора и выполняется за $O(2^n)$ (уточнить), поэтому описанный здесь алгоритм подойдет лишь для небольшого количества кластеров и признаков. 

# Practice!

У меня под рукой чисто случайно оказался старый датасет сервиса menu.by. В нем я нашел информацию о пользователях и их заказах. Я аггрегировал предпочтения клиентов простым скриптом и получил следующую таблицу

(Изображение таблицы)

Выше найдете следующие признаки
- active_days - количество дней активности
- avg_order - средний чек пользователя, выраженный в копейках BYN
- unique_items_per_order - среднее пунктов на один заказ
- unique_items_per_order - среднее количество уникальных пунктов на один заказ
- lifetime - время жизни пользователя сервиса

Все признаки (кроме lifetime) аггрегированы за последний месяц. Давайте посмотрим на их распределения. Разумеется, сперва логарифмируем значения некоторых признаков - иначе их распределения будут иметь длинный хвост, и мы ничего путного не увидим:

Вы слышите отдаленные крики? Это столбец lifetime = 0 требует, чтобы для него назначили отдельный признак. Добавляем в таблицу поле one_day_user - для случая, когда пользователь был активен только один день. 

Поскольку я исчерпал дневной лимит на препроцессинг признаков, начинаем кластеризовать. Используем православный k-means, чтобы получить долю ненависти со стороны DS-сообщества, и выделяем кластеры. После этого настает время для нашего супер-пупер-дупер объяснителя! Запускаем его для всех найденных кластеров и признаков датасета - стараемся найти интерпретацию, которая будет охватывать хотя бы 60% кластера (порог можно менять).

```python
explanation = ClustersExplanation(
    clusters, 
    features,
    thresholds=np.linspace(0, 0.4, 26)
)

explanation.fit(users_df)

explanation.score()
# 1.0 - значит, все пары кластеров различаются хотя бы одним правилом

pd.DataFrame(explanation.explain())
```
(Картинка с объяснением)

В таблице выше приведены правила, по которым кластеры можно отличить друг от друга. Для того, чтобы назвать правила, придется применить фантазию. Фрагмент кода ниже позволяет получить названия кластеров и интерпретацию каждого из правил в таблице

```python

rule_interpretation = [
    ('inactive', ''), # Пользователи в приложении меньше 1-го дня точно не являются активными
    ('low-spending', ''), # 19 BYN на момент сбора данных были равны почти 10 USD - не очень большая трата
    ('', 'mass-buying'), # Заказ, который содержит от 4-х предметов, можно назвать большим
    ('?', ''), # Часто ли вы составляете заказ из одного или двух пунктов? Вряд ли. Может, этим ребятам не предложили имбирь и палочки
    ('fresh', 'experienced'), # Пользователи, которые в нашем сервисе уже более полугода, а также новые клиенты
    ('returning', 'first-time') # Сегодняшняя порция клиентов, а также те, кто вернулся сюда снова
]

explanation.get_legend(rule_interpretation)
(Таблица с названиями)

explanation.get_cluster_names(rule_interpretation)
(Таблица с ...)
```

Такая визуализация уже даст значительно больше, чем цветные точки! Это ведь готовый набор действий для проведения кампании - например ... среди ... . 